In [2]:
import pandas as pd
import itertools
import os
import re
import time
from tqdm import tqdm

## Import Asian Tree Bank dataset (training)

In [49]:
import pandas as pd

splits = {'train': 'alt-en/train-00000-of-00001.parquet', 'validation': 'alt-en/validation-00000-of-00001.parquet', 'test': 'alt-en/test-00000-of-00001.parquet'}
df_treebank = pd.read_parquet("hf://datasets/mutiyama/alt/" + splits["train"])

'HTTPSConnectionPool(host='cdn-lfs.hf.co', port=443): Read timed out.' thrown while requesting GET https://huggingface.co/datasets/mutiyama/alt/resolve/main/alt-en/train-00000-of-00001.parquet
Retrying in 1s [Retry 1/5].


In [50]:
len(df_treebank)

17889

In [51]:
df_treebank.head()

,SNT.URLID,SNT.URLID.SNTID,url,status,value
0,46605,5273,http://en.wikinews.org/wiki/Swan_in_German_zoo_tests_positive_for_H5N1_virus,draft,"(S (S (NP (NP (DT A) (QT (DQL ``) (BASENP (ADJP (RB highly) (JJ pathogenic)) (NN strain)) (DQR ''))) (PP (IN of) (BASENP (DT the) (NNP H5N1) (NNP Avian) (NNP Flu) (NN virus)))) (VP (VBZ has) (VP (VBN been) (VP (VBN detected) (PP (IN in) (NP (BASENP (DT a) (JJ dead) (JJ Australian) (JJ black) (NN swan)) (COMMA ,) (S (VP (VBG floating) (PP (IN in) (NP (BASENP (DT a) (NN pond)) (VP (VBN located) (PP (IN at) (NP (BASENP (DT the) (NNP Dresden) (NNP Zoo)) (PP (IN in) (NP (BASENP (NNP Dresden)) (COMMA ,) (BASENP (NNP Germany))))))))))))))))) (PERIOD .))"
1,46605,5274,http://en.wikinews.org/wiki/Swan_in_German_zoo_tests_positive_for_H5N1_virus,draft,"(S (S (BASENP (JJ Conflicting) (NNS reports)) (VP (VBP say) (SBAR (IN that) (S (BASENP (DT the) (NN swan)) (VP (VBD was) (VP (VBN found) (PP (IN on) (NP (NP (BASENP (NNP Monday)) (COMMA ,) (BASENP (NNP Tuesday))) (CC or) (NNP Wednesday))))))))) (PERIOD .))"
2,46605,5275,http://en.wikinews.org/wiki/Swan_in_German_zoo_tests_positive_for_H5N1_virus,draft,(S (S (NP (BASENP (DT The) (NNP Friedrich) (NNP Loeffler) (NNP Institute)) (SBAR (WHNP (WDT which)) (S (VP (VBZ specializes) (PP (IN in) (BASENP (NN animal) (NNS diseases))))))) (VP (VBD confirmed) (BASENP (DT the) (NN test) (NNS results)))) (PERIOD .))
3,46605,5276,http://en.wikinews.org/wiki/Swan_in_German_zoo_tests_positive_for_H5N1_virus,draft,"(SINV (SINV (QT (DQL ``) (QTC (S (BASENP (PRP We)) (VP (MD can) (VP (VB confirm) (SBAR (IN that) (S (BASENP (DT this)) (VP (VBZ is) (BASENP (DT the) (ADJP (RB highly) (JJ pathogenic)) (NN strain)))))))) (COMMA ,)) (DQR '')) (SINV (VP (VBD said)) (NP (BASENP (DT a) (NN spokesperson)) (PP (IN for) (BASENP (DT the) (NN institute)))))) (PERIOD .))"
4,46605,5277,http://en.wikinews.org/wiki/Swan_in_German_zoo_tests_positive_for_H5N1_virus,draft,(S (S (BASENP (NNS Officials)) (VP (VP (VBP have) (VP (VP (VBN blocked) (PRT (RP off))) (NP (NP (NP (BASENP (DT an) (NN area)) (PP (IN of) (BASENP (NNP Dresden)))) (NP (NP (BASENP (QP (RB approximately) (NN 1.9)) (NNS miles)) (PRN (LRB -LRB-) (BASENP (NN 3km)) (RRB -RRB-))) (PP (IN from) (BASENP (DT the) (NN site))))) (SBAR (WHADVP (WRB where)) (S (BASENP (DT the) (NN swan)) (VP (VBD was) (VP (VBN found)))))))) (CC and) (VP (VBP have) (VP (VP (VP (VBN posted) (BASENP (NN observation) (NNS areas))) (PP (IN at) (NP (NP (BASENP (QP (RB approximately) (NN 6.2)) (NNS miles)) (PRN (LRB -LRB-) (BASENP (NN 10km)) (RRB -RRB-))) (PP (IN outside) (BASENP (DT the) (NN area)))))) (PP (IN as) (BASENP (DT a) (NN precaution))))))) (PERIOD .))


In [ ]:
# find row SNT.URLID.SNTID 5273 in df_treebank
row = df_treebank[df_treebank['SNT.URLID.SNTID'] == 5273]
row

,SNT.URLID,SNT.URLID.SNTID,url,status,value


In [14]:
splits_parallel = {'train': 'alt-parallel/train-00000-of-00001.parquet', 'validation': 'alt-parallel/validation-00000-of-00001.parquet', 'test': 'alt-parallel/test-00000-of-00001.parquet'}

df_parallel = pd.read_parquet("hf://datasets/mutiyama/alt/" + splits_parallel["train"]) 

In [15]:
df_parallel.head()

,SNT.URLID,SNT.URLID.SNTID,url,translation
0,80188,1,http://en.wikinews.org/wiki/2007_Rugby_World_C...,{'bg': 'ফ্রান্সের প্যারিসের পার্ক দি প্রিন্সেস...
1,80188,2,http://en.wikinews.org/wiki/2007_Rugby_World_C...,{'bg': 'অ্যান্ড্রিয়া মেসি চার মিনিটের মাথায় ট্...
2,80188,3,http://en.wikinews.org/wiki/2007_Rugby_World_C...,{'bg': 'প্রথমার্ধের বেশির ভাগ সময় ধরেই খেলা নি...
3,80188,4,http://en.wikinews.org/wiki/2007_Rugby_World_C...,{'bg': 'পর্তুগাল কখনোই হাল ছেড়ে দেয়নি ও ৩৩ মিন...
4,80188,5,http://en.wikinews.org/wiki/2007_Rugby_World_C...,{'bg': 'প্রথমার্ধে ইটালি ১৬-৫-এ এগিয়ে ছিল কিন্...


In [26]:
# unpack the dict in each row into two new columns
df_parallel['en'] = df_parallel['translation'].apply(lambda d: d['en'])
df_parallel['id'] = df_parallel['translation'].apply(lambda d: d['id'])

df_parallel.head()


,SNT.URLID,SNT.URLID.SNTID,url,translation,en,id
0,80188,1,http://en.wikinews.org/wiki/2007_Rugby_World_C...,{'bg': 'ফ্রান্সের প্যারিসের পার্ক দি প্রিন্সেস...,Italy have defeated Portugal 31-5 in Pool C of...,Italia berhasil mengalahkan Portugal 31-5 di g...
1,80188,2,http://en.wikinews.org/wiki/2007_Rugby_World_C...,{'bg': 'অ্যান্ড্রিয়া মেসি চার মিনিটের মাথায় ট্...,Andrea Masi opened the scoring in the fourth m...,Andrea Masi membuka skor di menit keempat deng...
2,80188,3,http://en.wikinews.org/wiki/2007_Rugby_World_C...,{'bg': 'প্রথমার্ধের বেশির ভাগ সময় ধরেই খেলা নি...,Despite controlling the game for much of the f...,Meskipun mengontrol jalannya pertandingan untu...
3,80188,4,http://en.wikinews.org/wiki/2007_Rugby_World_C...,{'bg': 'পর্তুগাল কখনোই হাল ছেড়ে দেয়নি ও ৩৩ মিন...,Portugal never gave up and David Penalva score...,Portugal tidak pernah menyerah dan David Penal...
4,80188,5,http://en.wikinews.org/wiki/2007_Rugby_World_C...,{'bg': 'প্রথমার্ধে ইটালি ১৬-৫-এ এগিয়ে ছিল কিন্...,Italy led 16-5 at half time but were matched b...,Italia memimpin 16-5 di babak pertama namun me...


In [ ]:
# df_treebank.to_parquet("data/train_treebank.parquet")
# df_parallel.to_parquet("data/train_parallel.parquet")

### Merge treebank with translation dataset

In [65]:
df_parallel = df_parallel.drop(columns=['translation'])

In [66]:
df_parallel = df_parallel.merge(df_treebank[['SNT.URLID.SNTID', 'value']], on='SNT.URLID.SNTID')

In [70]:
df_parallel.head()

,SNT.URLID,SNT.URLID.SNTID,url,en,id,value
0,80188,1,http://en.wikinews.org/wiki/2007_Rugby_World_Cup:_Italy_31_-_5_Portugal,"Italy have defeated Portugal 31-5 in Pool C of the 2007 Rugby World Cup at Parc des Princes, Paris, France.","Italia berhasil mengalahkan Portugal 31-5 di grup C dalam Piala Dunia Rugby 2007 di Parc des Princes, Paris, Perancis.","(S (S (BASENP (NNP Italy)) (VP (VBP have) (VP (VP (VP (VBN defeated) (BASENP (NNP Portugal))) (ADVP (RB 31-5))) (PP (IN in) (NP (BASENP (NNP Pool) (NNP C)) (PP (IN of) (NP (BASENP (DT the) (NN 2007) (NNP Rugby) (NNP World) (NNP Cup)) (PP (IN at) (NP (BASENP (NNP Parc) (FW des) (NNP Princes)) (COMMA ,) (BASENP (NNP Paris) (COMMA ,) (NNP France))))))))))) (PERIOD .))"
1,80188,2,http://en.wikinews.org/wiki/2007_Rugby_World_Cup:_Italy_31_-_5_Portugal,Andrea Masi opened the scoring in the fourth minute with a try for Italy.,Andrea Masi membuka skor di menit keempat dengan satu try untuk Italia.,(S (S (BASENP (NNP Andrea) (NNP Masi)) (VP (VP (VP (VBD opened) (BASENP (DT the) (NN scoring))) (PP (IN in) (BASENP (DT the) (JJ fourth) (NN minute)))) (PP (IN with) (NP (BASENP (DT a) (NN try)) (PP (IN for) (BASENP (NNP Italy))))))) (PERIOD .))
2,80188,3,http://en.wikinews.org/wiki/2007_Rugby_World_Cup:_Italy_31_-_5_Portugal,"Despite controlling the game for much of the first half, Italy could not score any other tries before the interval but David Bortolussi kicked three penalties to extend their lead.","Meskipun mengontrol jalannya pertandingan untuk sebagian besar dari setengah permainan, Italia tidak dapat menambah skor melalui try lainnya sebelum istirahat, namun David Bortolussi menendang tiga penalti untuk memperlebar keunggulan mereka.","(S (S (S (PP (IN Despite) (S (VP (VP (VBG controlling) (BASENP (DT the) (NN game))) (PP (IN for) (NP (BASENP (JJ much)) (PP (IN of) (BASENP (DT the) (JJ first) (NN half)))))))) (COMMA ,) (S (BASENP (NNP Italy)) (VP (VP (MD could) (RB not)) (VP (VP (VB score) (BASENP (DT any) (JJ other) (NN tries))) (PP (IN before) (BASENP (DT the) (NN interval))))))) (CC but) (S (BASENP (NNP David) (NNP Bortolussi)) (VP (VP (VBD kicked) (BASENP (NN three) (NNS penalties))) (S (VP (TO to) (VP (VB extend) (BASENP (PRPD their) (NN lead)))))))) (PERIOD .))"
3,80188,4,http://en.wikinews.org/wiki/2007_Rugby_World_Cup:_Italy_31_-_5_Portugal,"Portugal never gave up and David Penalva scored a try in the 33rd minute, providing their only points of the match.",Portugal tidak pernah menyerah dan David Penalva mencetak skor melalui try di menit ke-33 yang menjadi skor akhir mereka.,"(S (S (S (BASENP (NNP Portugal)) (VP (ADVP (RB never)) (VP (VBD gave) (PRT (RP up))))) (CC and) (S (BASENP (NNP David) (NNP Penalva)) (VP (VP (VP (VBD scored) (BASENP (DT a) (NN try))) (PP (IN in) (BASENP (DT the) (JJ 33rd) (NN minute)))) (COMMA ,) (S (VP (VBG providing) (NP (BASENP (PRPD their) (JJ only) (NNS points)) (PP (IN of) (BASENP (DT the) (NN match))))))))) (PERIOD .))"
4,80188,5,http://en.wikinews.org/wiki/2007_Rugby_World_Cup:_Italy_31_-_5_Portugal,Italy led 16-5 at half time but were matched by Portugal for much of the second half.,Italia memimpin 16-5 di babak pertama namun mendapat perlawanan dari portugal di babak kedua.,(S (S (BASENP (NNP Italy)) (VP (VP (VP (VBD led) (ADVP (RB 16-5))) (PP (IN at) (BASENP (JJ half) (NN time)))) (CC but) (VP (VBD were) (VP (VP (VBN matched) (PP (IN by) (BASENP (NNP Portugal)))) (PP (IN for) (NP (BASENP (JJ much)) (PP (IN of) (BASENP (DT the) (JJ second) (NN half))))))))) (PERIOD .))


In [ ]:
# df_parallel.to_parquet("data/train_merged.parquet", index=False)

## Check columns that contains preposition